# Tumor Levelset Extraction

The purpose of this notebook is to create an HTML table visualizing the meshes corresponding to the levelsets of the tumor segmentation boundaries

## Step 1: Load Data

In [ ]:
import nibabel as nib
import numpy as np 
import matplotlib.pyplot as plt
from skimage import measure # For marching cubes
import polyscope as ps # For mesh display
import pandas as pd
import os
import glob

def load_dictionary(metadata_path):
    df = pd.read_csv(metadata_path)
    data = {}
    for index, row in df.iterrows():
        patient_id = row["ID"]
        patient_id = "M-0".join(patient_id.split("M-")) #File paths have an extra 0 in ID
        data[patient_id] = row
    del data["UCSF-PDGM-0541"] # Skip Patient 541 because segmentation file is empty
    return data

In [ ]:
def argsort(seq):
    return np.array(sorted(range(len(seq)), key=seq.__getitem__), dtype=int)

metadata_path = "Data/UCSF-PDGM-metadata_v2.csv"
all_data_path = "Data/UCSF-PDGM-v3"
data = load_dictionary(metadata_path) 

patients = list(data.keys())
diagnosis = [data[p]["Final pathologic diagnosis (WHO 2021)"] for p in patients]
dead = np.array([data[p]["1-dead 0-alive"] for p in patients])
# Sort by dead/alive first, then by diagnosis
idx = np.argsort(dead)
idx = idx[argsort([diagnosis[i] for i in idx])]
patients = [patients[i] for i in idx]


## Step 2: Extract meshes from all patients at 3 different isolevels (edema, main tissue, necrotic tissue)

In [ ]:
iso_names = {1:"Necrotic", 2:"Edema", 4:"Main Tumor"} # What the labels actually mean
iso_levels = [2, 4, 1] # Column order of the labels
for i, patient in enumerate(os.listdir(all_data_path)):
    if i%10 == 0:
        print(".", end="")
    patient_folder_path = os.path.join(all_data_path, patient)
    patient = patient[:-6]
    if patient in data:
        tumor_seg_path = patient_folder_path + "/" + patient_folder_path[-20:-6] + "_tumor_segmentation.nii.gz"
        tumor_seg_nifti = nib.load(tumor_seg_path)
        tumor_seg_mat = tumor_seg_nifti.get_fdata()
        
        #Confirm the affine matrix is indeed non-scaling + arbitrary translation vector. If not report out.
        affine_mat = tumor_seg_nifti.affine[:3, :3]
        if affine_mat[0,0] == -1 and affine_mat[1,1] == -1 and affine_mat[2,2] == 1:
            for k, level in enumerate(iso_levels):
                try:
                    binary = tumor_seg_mat==level
                    data[patient]["count{}".format(k)] = np.sum(binary)
                    data[patient]["V{}".format(k)], data[patient]["T{}".format(k)], _, _ = measure.marching_cubes(binary, 0.5)
                except:
                    #print("Empty mesh", patient, iso_names[iso_levels[k]])
                    data[patient]["V{}".format(k)] = np.array([])
                    data[patient]["T{}".format(k)] = np.array([], dtype=int)
        else:
            print("Nonstandard scaling", patient, affine_mat)

## Step 3: Visualize Data

In [ ]:
ps.init()

center = np.array(list(tumor_seg_mat.shape))/2
camera_pos = np.zeros(3)
camera_pos[0] = center[0]
camera_pos[1] = center[1]
camera_pos[2] = -center[2]*4

table_style = """
 table, th, td {
  border: 1px solid;
}"""
display_names = [iso_names[i] for i in iso_levels]
fout = open("viz/index.html", "w")
fout.write("<html><head><style>{}</style></head>\n<body>\n<table><tr><td><h2>Patient Info</h2></td>".format(table_style))
fout.write("<td><h2>{}</h2></td><td><h2>{}</h2></td><td><h2>{}</h2></td></tr>\n".format(*tuple(display_names)))
for p in patients:
    d = data[p]
    diagnosis = d["Final pathologic diagnosis (WHO 2021)"]
    dead = ["alive", "dead"][d["1-dead 0-alive"]]
    OS = d["OS"]
    if not "Glioblastoma" in diagnosis:
        continue
    if not "V0" in d:
        print(p, "does not have mesh info")
        continue
    ps.remove_all_structures()
    meshes = []
    for k in range(3):
        m = None
        if d["V%i"%k].size > 0:
            m = ps.register_surface_mesh("tumor%i"%k, d["V%i"%k], d["T%i"%k])
        meshes.append(m)
    ps.look_at(camera_pos, center)
    #ps.set_view_projection_mode("orthographic")

    fout.write("<tr>")
    fout.write("<td><h2>{}</h2><h2>{}</h2><h2>{}</h2><h2>OS: {}</h2>".format(p, diagnosis, dead, OS))
    for k in range(3):
        for m in meshes:
            if m:
                m.set_enabled(False)
        if meshes[k]:
            meshes[k].set_enabled(True)
        filename = "viz/{}_{}.jpg".format(p, k)
        ps.screenshot(filename=filename)
        count = d["count%i"%k]
        V = d["V%i"%k]
        T = d["T%i"%k]
        fout.write("<td><p>Count {}, {} Vertices, {} Triangles</p><img src=\"{}_{}.jpg\"></td>".format(count, V.shape[0], T.shape[0], p, k))
    fout.write("</tr>\n")
    
    fout.flush()
fout.close()
    
# Downscale all images to 50% and autocrop them.  Requires imagemagick
import subprocess
subprocess.call(["mogrify", "-resize", "50%", "viz/*.jpg"])
subprocess.call(["mogrify", "-trim", "viz/*.jpg"])